In [ ]:
import pandas as pd
import altair as alt
from altair import datum
from vega import Vega
violence = pd.read_csv("E:\\SEED\\OneDrive\\Msc. Biostatistics\\Level Two\\Second Semester\\Visualisation\\Datasets\\2016-01-01-2018-01-01-Western_Africa-Nigeria.csv")
violence = violence[violence.year != 2018]

In [ ]:
chart_1 = alt.Chart(violence).mark_circle().encode(
    x = alt.X('month(event_date):T', title = 'Month'),
    y = alt.Y('admin1:N' , title = 'Administrative Unit'),
    size = 'mean(fatalities):Q',
   
    color=alt.Color('admin1:N',legend=None),
    tooltip = ['admin1','mean(fatalities):Q']
  ).  properties(
    height = 500,
    width = 400,
    title = 'Mean Fatalities over the month for different administattive Units'
  )
chart_1

In [ ]:
#####PLOTONE Geo Map #########
brush = alt.selection_interval()  # selection of type "interval"
chart_2 = alt.Chart(violence).mark_circle(color='red').encode(
    x = 'longitude:Q',
    y = 'latitude:Q',
    size = 'sum_fatality:Q',
    opacity = 'sum_fatality:Q',
    tooltip = ["location:N","sum_fatality:Q"]
  ).transform_aggregate(
    sum_fatality='sum(fatalities)',
    groupby=["location","longitude","latitude"]).properties(
    title="Total deaths by Location",
    height = 200,
    width = 350
  ).add_selection(brush)

chart_21 = alt.Chart(violence).mark_circle(color='red').encode(
    x = 'longitude:Q',
    y = 'latitude:Q',
    size = 'total_attack:Q',
    opacity = 'total_attack:Q',
    tooltip = ["location:N","total_attack:Q"]
  ).transform_aggregate(
    total_attack='count()',
    groupby=["location","longitude","latitude"]).properties(
    title="Total deaths by Location",
    height = 200,
    width = 350
  ).add_selection(brush)
chart_2 |chart_21

In [ ]:
######Plot 3- Event type by year########
chart_3 = alt.Chart(violence).mark_bar().encode(
    x = 'total_fatality:Q',
    y = alt.Y('event_type:N',sort='-x'),
    color = 'year:N',
    tooltip = ['year:N','total_fatality:Q']
  ).transform_aggregate(
    total_fatality ='sum(fatalities)',
    groupby = ["year","event_type"]
  ).properties(
    width= 300,
    height = 200,
    title = "Total fatalities by Event type and Year"
  )
chart_3

In [ ]:
#####Plot 4: monthly total death trend##########
chart_4 = alt.Chart(violence).mark_line().encode(
    x = alt.X('month:T',title = 'Event month', axis=alt.Axis(format='%b')),
    y = alt.Y('avg_fatality:Q',title = 'Average fatality'),
    color = 'year:N',
    tooltip = ['year:N','avg_fatality:Q']
  ).transform_timeunit(
    month='month(event_date)'
  ).transform_aggregate(
    avg_fatality = 'mean(fatalities)',
    groupby = ["month","year"] 
  ).properties(
    width = 400,
    height = 200,
    title = 'Average monthly death trend by Year'
  )
chart_4

In [ ]:
####Plot 5:daily mean death#######

selection = alt.selection_multi(fields=['event_type'], bind='legend')

chart_5 = alt.Chart(violence).mark_point().encode(
    x = alt.X('day:T',title = "Day of the week",axis = alt.Axis(format = '%a')),
    y = alt.Y('avg_fatality:Q', title = "Mean Fatality"),
    color = 'event_type:N',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    size = 'avg_fatality:Q',
    tooltip = ['event_type:N','avg_fatality:Q']
  ).transform_timeunit(
    day = 'day(event_date)'
  ).transform_aggregate(
    avg_fatality = 'mean(fatalities)',
    groupby = ["day","event_type"]
).properties(
    width = 300,
    height = 300
  ).add_selection(selection)
chart_5

In [ ]:
###Cummulative count#####
input_dropdown = alt.binding_select(options=sorted(['Nasarawa','Benue', 'Rivers', 'Kwara', 'Borno', 'Bayelsa',
       'Adamawa', 'Kaduna', 'Yobe', 'Taraba', 'Federal Capital Territory',
       'Oyo', 'Ekiti', 'Delta', 'Lagos', 'Osun', 'Cross River', 'Jigawa',
       'Kebbi', 'Imo', 'Anambra', 'Ogun', 'Enugu', 'Ondo', 'Niger',
       'Kogi', 'Edo', 'Akwa Ibom', 'Plateau', 'Ebonyi', 'Kano', 'Katsina',
       'Abia', 'Zamfara', 'Bauchi', 'Sokoto', 'Gombe']))
selection = alt.selection_single(fields=['admin1'], bind=input_dropdown, name='Administrative Unit')

color = alt.condition(selection,
                    alt.Color('Origin:N', legend=None),
                    alt.value('lightgray'))

chart_6a = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['actor1']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
    (alt.datum.rank < 11)
).mark_bar().encode(
    x="total_dead:Q",
     y = alt.Y('actor1:N',sort='-x',title='Actors'),
    tooltip = ['total_dead:Q']
).properties(title = 'Ten most deadly groups',height=150, width=300)

chart_6b = chart_6a.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='total_dead:Q'
)

chart_6 = (chart_6a + chart_6b).add_selection(selection).transform_filter(selection)
chart_6


In [ ]:
chart_6c = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['event_type','year']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
     (alt.datum.rank < 11 & alt.datum.year !=2018)
).mark_bar().encode(
    x=alt.X('total_dead:Q', stack='zero'),
     y = alt.Y('event_type:N',sort='-x',title='Event Type'),
    tooltip = ['total_dead:Q'],
    color='year:N'
).properties(title = 'Dead distribution By Event Type',height=150, width=300)

chart_6d = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['event_type','year']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
    (alt.datum.rank < 11 & alt.datum.year !=2018)
).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('total_dead:Q', stack='zero'),
     y = alt.Y('event_type:N',sort='-x',title='Event Type'),
    tooltip = ['year','total_dead:Q'],
    text=alt.Text('total_dead:Q')
).properties(title = 'Dead distribution By Event Type',height=150, width=300)
chart_61 = (chart_6c + chart_6d).add_selection(selection).transform_filter(selection)
chart_61



In [ ]:
chart_6e = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['admin1','year']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
    (alt.datum.rank < 11)
).mark_bar().encode(
    y="total_dead:Q",
     x = alt.X('year:N',sort='-y'),
    tooltip = ['total_dead:Q'],
    color = 'year:N',
    column = 'admin1:N'
).properties(title = 'Top most affected Regions',height=150)

chart_6e



In [ ]:
####Dead by Event_type Over the months####
chart_7a = alt.Chart(violence).mark_line().encode(
    x = alt.X('month(event_date):T', title = 'Month'),
    y='sum(fatalities):Q',
    color = 'year:N'
).properties(height=150, width=300)
chart_7b = chart_7a.mark_text(
    baseline='top',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='sum(fatalities):Q'
)
chart_7 = (chart_7a + chart_7b).add_selection(selection).transform_filter(selection)
chart_7



In [ ]:
chart_8 = Vega({
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A word cloud visualization depicting Vega research paper abstracts.",
  "width": 800,
  "height": 400,
  "padding": 0,

  "data": [
    {
      "name": "table",
      "url": "https://gist.githubusercontent.com/ivombi/bc443fffdb88824b89d7241e9ef447c5/raw/417e83cd2d7aee96c5819ef849e80063c5a7a312/violence.csv", "format":{"type":"csv"},
      "transform": [
        {
          "type": "countpattern",
          "field": "notes",
          "case": "upper",
          "pattern": "[\\w']{5,}",
          "stopwords": "(Nigeria|Against|during|least|clear|later|place|hours|dated|seven|eight|began|unknown|known|unpaid|following|office|staff|along|three|labour||among|while|before|several|about|other|second|others|which|between|under|reportedly|after|their)"
        },
        {
          "type": "formula", "as": "angle",
          "expr": "[-45, 0, 45][~~(random() * 3)]"
        },
        {
          "type": "formula", "as": "weight",
          "expr": "if(datum.text=='VEGA', 600, 300)"
        },
        {"type": "filter", "expr": "datum.count > 50"}
      ]
    }
  ],

  "scales": [
    {
      "name": "color",
      "type": "ordinal",
      "domain": {"data": "table", "field": "text"},
      "range": ["#d5a928", "#652c90", "#939597"]
    }
  ],
    "title":{"text":"Most Used words","fontSize":30,"fontStyle": {"value": "italic"}},

  "marks": [
    {
      "type": "text",
      "from": {"data": "table"},
      "encode": {
        "enter": {
          "text": {"field": "text"},
          "align": {"value": "center"},
          "baseline": {"value": "alphabetic"},
          "fill": {"scale": "color", "field": "text"}
        },
        "update": {
          "fillOpacity": {"value": 1}
        },
        "hover": {
          "fillOpacity": {"value": 0.5}
        }
      },
      "transform": [
        {
          "type": "wordcloud",
          "size": [800, 400],
          "text": {"field": "text"},
          "rotate": {"field": "datum.angle"},
          "font": "Helvetica Neue, Arial",
          "fontSize": {"field": "datum.count"},
          "fontWeight": {"field": "datum.weight"},
          "fontSizeRange": [12, 56],
          "padding": 2
        }
      ]
    }
  ]
},violence)
chart_8

In [ ]:
chart_9 = Vega({
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A basic donut chart example.",
  "width": 200,
  "height": 100,
  
  "autosize": "pad",

  "signals": [
    {"name": "startAngle", "value": 0},
    {"name": "endAngle", "value": 6.29},
    {"name": "sort", "value": True},
    {"name": "padAngle", "value": 0},
    {"name": "strokeWidth", "value": 2}
  ],
  

  "data": [
    {
      "name": "table",
      "url": "https://gist.githubusercontent.com/ivombi/bc443fffdb88824b89d7241e9ef447c5/raw/417e83cd2d7aee96c5819ef849e80063c5a7a312/violence.csv", "format":{"type":"csv"},
      "transform": [
        {
  "type": "aggregate",
  "fields": ["fatalities"],
  "ops": ["sum"],
  "as": ["Total_Fatalities"],
  "groupby":["event_type"]
},
        {
          "type": "pie",
          "field": "Total_Fatalities",
          "startAngle": {"signal": "startAngle"},
          "endAngle": {"signal": "endAngle"},
          "sort": True
        }
      ]
    }
  ],

  "scales": [
    {
      "name": "color",
      "type": "ordinal",
      "domain": {"data": "table", "field": "event_type"},
      "range": {"scheme": "category20"}
    }
  ],
  "legends": [
  {
    "fill": "color"
  }
],
"title": {
  "text": "Even Type Distribution"},
  "marks": [
    {
      "type": "arc",
      "from": {"data": "table"},
      "encode": {
        "enter": {
          "fill": {"scale": "color", "field": "event_type"},
          "x": {"value": 100},
          "y": {"value": 100},
          "tooltip":{"field":"Total_Fatalities"}
        },
        "update": {
          "startAngle": {"field": "startAngle"},
          "endAngle": {"field": "endAngle"},
          "padAngle": {"value": 0},
          "innerRadius": {"value": 60},
          "outerRadius": {"value": 100},
          "cornerRadius": {"value": 0}
        }
      }
    }
  ]
},violence)
chart_9

In [ ]:
chart_6c = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['event_type','year']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
     (alt.datum.rank < 11 & alt.datum.year !=2018)
).mark_bar().encode(
    x=alt.X('total_dead:Q', stack='zero'),
     y = alt.Y('event_type:N',sort='-x',title='Event Type'),
    tooltip = ['total_dead:Q'],
    color='year:N'
).properties(title = 'Dead distribution By Event Type',height=150, width=300)

chart_6d = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['event_type','year']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
    (alt.datum.rank < 11 & alt.datum.year !=2018)
).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('total_dead:Q', stack='zero'),
     y = alt.Y('event_type:N',sort='-x',title='Event Type'),
    tooltip = ['year','total_dead:Q'],
    text=alt.Text('total_dead:Q')
).properties(title = 'Dead distribution By Event Type',height=150, width=300)
chart_6c + chart_6d

In [ ]:
chart_6e = alt.Chart(violence).transform_aggregate(
    total_dead='sum(fatalities):Q',groupby=['admin1','year']
).transform_window(
    rank='rank(total_dead)',
    sort=[alt.SortField('total_dead', order='descending')]
).transform_filter(
    (alt.datum.rank < 11)
).mark_bar().encode(
    y="total_dead:Q",
     x = alt.X('year:N'),
    tooltip = ['total_dead:Q'],
    color = 'year:N',
    column = 'admin1:N'
).properties(title = 'Dead distribution By Region',height=150)
chart_6e